<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [478]:
import pandas as pd
import psycopg2

In [480]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [481]:
# текст запроса
query_3_1 = f'''
select
    count(*) vacancies_count
from
    public.vacancies
'''

In [482]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [483]:
# текст запроса
query_3_2 = f'''
select
    count(*) employers_count
from
    public.employers
'''

In [484]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [485]:
# текст запроса
query_3_3 = f'''
select
    count(*) areas_count
from
    public.areas
'''

In [486]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [487]:
# текст запроса
query_3_4 = f'''
select
    count(*) industries_count
from
    public.industries
'''

In [488]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,industries_count
0,294


***

#### Выводы по предварительному анализу данных
Соединение с базой данных было установлено, проведен предварительный анализ данных. Предварительный анализ показал, что таблицы с вакансиями, работодателями, регионами и со сферами деятельности не пустые, содержат по 49197, 23501, 1362, 294 записей соответственно.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [489]:
# текст запроса
query_4_1 = f'''
select
    a.name, count(*) cnt
from
    public.vacancies v
join
    public.areas a
        on a.id=v.area_id
group by
    a.id
order by
    cnt desc
'''

In [490]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [491]:
# текст запроса
query_4_2 = f'''
select
    count(*) vacancies_count
from
    public.vacancies
where
    salary_from is not null or
    salary_to is not null
'''

In [492]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,vacancies_count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [493]:
# текст запроса
# к типу int привел намеренно, чтобы результат был целые числа (чтобы было красиво, без .0 в конце числа :) )
query_4_3 = f'''
select
    round(avg(salary_from))::int avg_salary_from,
    round(avg(salary_to))::int avg_salary_to
from
    public.vacancies
where
    salary_from is not null or
    salary_to is not null
'''

In [494]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [495]:
# текст запроса
query_4_4 = f'''
select
    schedule,
    employment,
    count(*) vacancies_count
from
    public.vacancies
group by
    schedule,
    employment
order by
    vacancies_count desc
'''

In [496]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [497]:
# текст запроса
query_4_5 = f'''
select
    experience,
    count(*) vacancies_count
from
    public.vacancies
where
    experience is not null
group by
    experience
order by
    vacancies_count
'''

In [498]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

#### Выводы по детальному анализу вакансий
Детальный анализ вакансий показал:
    <li>Большинство вакансий сосредоточено в крупных городах, таких как: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы и др.</li>
    <li>У около половины вакансий не указаны зарплатные вилки</li>
    <li>Средние значения зарплатной вилки составляют от 71065 до 110537</li>
    <li>Подавляющее количество вакансий связана с полной занятостью. Топ 3 сочетаний типа трудоустройства + рабочего графика: полная занятость + полным рабочим днем с количеством вакансий 35367, затем идет полная занятость с удаленным рабочим графиком с количеством вакансий 7802, после идет полная занятость с гибким графиком с количеством вакансий 1593.
    <li>В около половине случаев ищут сотрудников с опытом работы от 1 года до 3 лет -- 26152 вакансий, затем идет группа вакансий с опытом работы от 3 до 6 лет -- 14511 вакансий, потом идет группа вакансий с поиском сотрудников без опыта работы -- 7197 вакансий, замыкает группа вакансий с опытом работы более 6 лет -- 1337 вакансий.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [499]:
# текст запроса
query_5_1 = f'''
select
    e.name,
    count(*) vacancies_count
from
    public.vacancies v
join
    public.employers e
        on e.id=v.employer_id
group by
    e.id
order by
    vacancies_count desc
'''

In [500]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14901,Евсеев Сергей Владимирович,1
14902,Кадровые технологии,1
14903,ЛОМБАРД 24,1
14904,Авто-Компонент ТД,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [501]:
# текст запроса

# Запрос в лоб, который выполняется 40-60 секунд
# query_5_2 = f'''
# select
#     a.name,
#     count(distinct e.id) employers_count,
#     count(distinct v.id) vacancies_count
# from
#     public.areas a
# left join
#     public.vacancies v
#         on v.area_id=a.id
# left join
#     public.employers e
#         on e.area=a.id
# group by
#     a.id
# order by
#     vacancies_count desc,
#     employers_count
# '''

# Оптимизированный запрос с двумя подзапросами (групировки в подзапросах делать выгоднее, чем делать группировку в "большой" таблице)
# Планы запросов не копал, применил только лишь логику :)
# Этот запрос выполняется за 0.1 секунды
query_5_2 = f'''
select
    a.name,
    coalesce(e.employers_count, 0) employers_count, -- подставляем нули у регионов в которых не представленны работодатели
    coalesce(v.vacancies_count, 0) vacancies_count  -- подставляем нули у регионов в которых нет вакансий
from
    public.areas a
left join (
        select
            area_id,
            count(*) vacancies_count
        from
            public.vacancies
        group by
            area_id
    ) v
        on v.area_id=a.id
left join (
        select 
            area,
            count(*) employers_count
        from
            public.employers
        group by
            area
    ) e
        on e.area=a.id
order by
    vacancies_count desc,
    employers_count
 '''

In [502]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,name,employers_count,vacancies_count
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Новосибирск,573,2006
4,Алматы,721,1892
...,...,...,...
1357,Ростовская область,18,0
1358,Краснодарский край,19,0
1359,Московская область,75,0
1360,Казахстан,207,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [503]:
# текст запроса
query_5_3 = f'''
select
    e.name,
    count(distinct v.area_id) areas_count
from
    public.employers e
left join
    public.vacancies v
        on v.employer_id=e.id
group by
    e.id
order by
    areas_count desc
'''

In [504]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [505]:
# текст запроса
query_5_4 = f'''
select
    count(*) employers_count
from
    public.employers
where
    id not in
        (
        select
            employer_id
        from
            employers_industries
        )
'''

In [506]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,employers_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [507]:
# текст запроса
query_5_5 = f'''
select
    e.name,
    count(*) industries_count
from
    public.employers e
join
    public.employers_industries i
        on i.employer_id=e.id
group by
    e.id
having
    count(*) = 4
order by
    name
'''

In [508]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,name,industries_count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1133,ЮРИОН,4
1134,ЮТИП Технологии,4
1135,ЯКласс,4
1136,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [509]:
# текст запроса

# Если исходить из того, что целосность базы данных нем не известна, то в запросе соединяем все таблицы
# query_5_6 = f'''
# select 
#     count(*) employers_count
# from
#     public.employers e
# join
#     public.employers_industries ei
#         on ei.employer_id=e.id
# join
#     public.industries i
#         on i.id=ei.industry_id
# where
#     i.name='Разработка программного обеспечения'
# '''

# Если целосность базы данных гарантируется (например, через внешние ключи и каскадные обновления/удаления),
# то тогда таблицу public.employers можно не использовать и сделать так:
query_5_6 = f'''
select
    count(*) employers_count
from
    public.employers_industries ei
join
    public.industries i
        on i.id=ei.industry_id
where
    i.name='Разработка программного обеспечения'
'''

In [510]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,employers_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [511]:
# код для получения списка городов-милионников
import requests 
from bs4 import BeautifulSoup

response = requests.get('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
page = BeautifulSoup(response.text, 'html.parser')
# Выбираем из документа вторую таблицу и в таблице выбираем первый уровень тега "a" внутри ячейки "td"
tda = page.select('div table:nth-of-type(2) tbody tr td > a')
mcities = [t.text for t in tda]
display(mcities)

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [512]:
# Часть запросов с таблицами и условиями фильтрации

# Вот такой вариант запроса очень плох тем, что мы не контролируем параметры которые в тело запроса встают и тут потенциально возможна sql-инъекция
# part_of_5_7 = f'''
# from
#     public.employers e
# left join
#     public.vacancies v
#         on v.employer_id=e.id
# left join
#     public.areas a
#         on v.area_id=a.id
# where
#     e.name='Яндекс' and
#     a.name in ('{"','".join(mcities)}')
# '''

# Такой вариант с параметрами и передачей их через предназначенный для этого механизм позволит правильно экранировать переменные и
# позволит избежать sql-инъекции
part_of_5_7 = f'''
from
    public.employers e
left join
    public.vacancies v
        on v.employer_id=e.id
left join
    public.areas a
        on v.area_id=a.id
where
    e.name='Яндекс' and
    a.name in %(mcities)s
'''

# текст запроса
query_5_7 = f'''
select
    a.name,
    count(*) cnt
{part_of_5_7}
group by a.id

union all

select
    'Total',
    count(*)
{part_of_5_7}
order by cnt
'''

In [513]:
# Выполнение запроса с параметрами
df = pd.read_sql_query(query_5_7, connection, params={'mcities': tuple(mcities)})
df

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

#### Выводы по анализу работодателей
Анализ работодателей показал:
<li>Яндекс является самым крупным по количеству вакансий -- 1933 вакансии, затем идут: Ростелеком -- 491, Тинькофф -- 444, Сбер -- 428, Газпром нефть -- 331. Ожидаемо, что топ ко количеству вакансий на ресурсе HH (площадка для it-специалистов) занимают крупные связанные it-сектором напрямую компании.</li>
<li>Крупнейшими городами по числу вакансий и работодателей являются крупнейшие города России и ближайших стран СНГ. Топ 5 крупнейших городов по количеству вакансий и работодателей: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Здесь также все ожидаемо, ведь крупные региональные центры/столицы стран концентрируют в себе большее количество различных крупных и мелких бизнесов.</li>
<li>В процессе анализа был выявлен топ 5 работодателей по количеству регионов в которых открыты вакансии: Яндекс -- 181 регион, Ростелеком -- 152 региона, Спецремонт -- 116 регионов, Поляков Денис Иванович -- 88 регионов. Здесь также все ожидаемо -- крпным компаниям необходимы сотрудники в различных регионах. Единственное здесь выбивается "Поляков Денис Иванович", рассмотрим этого работодателя ниже.</li>
<li>Около 35% работодателей не указали сферу деятельности.</li>
<li>Около 15% работодателей одной из сфер деятельности указали "Разработка программного обеспечения"</li>
<li>Около 25% вакансий Яндекса представлены в городах миллионниках России. Топ 5 городов: Москва -- 54 вакансии, Санкт-Петербург -- 42 вакансии, Екатеринбург -- 39 вакансий, Нижний Новгород -- 36 вакансий, Новосибирск -- 35 вакансий.</li>


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [514]:
# текст запроса
query_6_1 = f'''
select
    count(*) vacancies_count
from
    public.vacancies 
where
    lower(name) like '%данн%' or
    lower(name) like '%data%'
'''

In [515]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,vacancies_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [516]:
# Часть запроса которую будем использовать дальше в заданиях
from_and_where_for_ds = '''
from
    public.vacancies
where
    (
        lower(name) like '%data scientist%' or
        lower(name) like '%data science%' or
        lower(name) like '%исследователь данных%' or
        (
            name like '%ML%' and
            name not like '%HTML%'
        ) or
        lower(name) like '%machine learning%' or
        lower(name) like '%машинн%обучен%%'
    )
'''

# текст запроса
query_6_2 = f'''
select
    count(*) vacancies_count
{from_and_where_for_ds}
    and (
        lower(name) like '%junior%' or
        experience='Нет опыта' or
        employment='Стажировка'
    )
'''

In [517]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,vacancies_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [518]:
# текст запроса
query_6_3 = f'''
select
    count(*) vacancies_count
{from_and_where_for_ds}
    and (
        key_skills like '%SQL%' or
        lower(key_skills) like '%postgres%'
    )
'''

In [519]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,vacancies_count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [520]:
# текст запроса
query_6_4 = f'''
select
    count(*) vacancies_count
{from_and_where_for_ds}
    and lower(key_skills) like '%python%'
'''

In [521]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,vacancies_count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [522]:
# Из-за того, что ключевые навыки разделены символом табуляции можно с навыках вырезать этот символ и посчитать разницу между
# исходной длиной строки с навыками и строкой навыков с вырезанными символами табуляции, тем самым узнав сколько было в строке разделителей (символов табуляции)
# Количество ключевых навыков в этом случае будет равно количеству разделителей (табуляций) + 1

# текст запроса
query_6_5 = f'''
select
    round(avg(length(key_skills) - length(replace(key_skills, '\t', '')) + 1), 2) key_skills_count
{from_and_where_for_ds}
'''

In [523]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,key_skills_count
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [524]:
# текст запроса
# К int привел намеренно, чтобы в результате получились целые числа без дробной части
# Зесть, для примера (чтобы показать владение этой возможностью, но я предпочитаю комментарии писать не в sql-запросах :) ), вставил комментарий в sql-запрос.
query_6_6 = f'''
select
    experience,
    
    -- Всегда считаем сумму двух показателей вилки, если не задано минимальное значением, то вместо него берется максимальное и 
    -- наоборот когда не задано максимальное, то вместо него берется минимальное. Затем для получения среднего делим полученную сумму на 2,
    -- соответственно получеем либо среднее между минимальным и максимальным значением, либо одно из двух не пустых значений.
    
    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2))::int salary
{from_and_where_for_ds}
    and (
        salary_from is not null or
        salary_to is not null
    )
group by
    experience
'''

In [525]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,salary
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

#### Выводы по предметному анализу
Предметный анализ показал:
<li>3.5% вакансий связаны с данными</li>
<li>Имеется всего 51 вакансия для начинающего дата-саентиста</li>
<li>201 вакансия для дата-саентистов со знанием SQL или Postgres</li>
<li>351 вакансия для дата-саентиста со знаничем Python. Знание Python встречается чаще, чем знание SQL или Postgres.</li>
<li>Среднее количество навыков для дата-саентиста в вакансиях составляет 6.41</li>
<li>Максимальный средний уровень раработной платы 243115 для дата-саентиста придставлен в группе вакансий в которой требуются дата-саентисты с опытом от 3 до 6 лет. Далее идут дата-саентисты с опытом от 1 года до 3 лет со средней заработной платой 139675, замыкают список дата-саентисты без опыта работы со средней заработной платой 74643.</li>

# Общий вывод по проекту

Исследование вакансий из HeadHunter позволило выявить крупнейших работодаталей и крупнейшие регионы по количеству вакансий связанных с it-сферой. Лидерами среди работодателей являются Яндекс, Тинькофф, Сбер и другие. Лидерами среди регионов являются Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Проведенный анализ позволил выявить среднии размеры предлагаемых заработных плат в зависимости от опыта кондидата, позволил оценить рынок и востребованность в специалистах дата-саентистов, очертить зарплатные ожидания дата-саентистов.

Ниже преведено дополнительное исследование некоторых аспектов вакансий.

В продолжении исследования необходимо будет выявить топ 5-10 работодателей по размеру предлагаемой заработной платы в регионе Москва имеющих вакансии дата-саентистов.
Также в продолжении необходимо построить различные визуализации данных из резюме, например, построить и оценить распределения предлагаемых заработных плат, что позволит более точно оценить реальную картину с предлагаемыми заработными платами.

# Юнит 7. Дополнительное исследование

1. Рассмотрим количество вакансий без указаний вилки заработной платы и средние зарплатные вилки в разрезе требуемого опыта работы.

In [526]:
query_7_1 = f'''
select
    experience,
    count(*) vacancies_count, -- общее количество вакансий
    sum((salary_from is null and salary_to is null)::int) vacancies_no_salary_count, -- количество вакансий без вилки ЗП
    round(100 * sum((salary_from is null and salary_to is null)::int) / count(*)) vacancies_no_salary_percent, --  % вакансий без вилки ЗП
    round(avg(salary_from))::int avg_salary_from,
    round(avg(salary_to))::int avg_salary_to
from
    public.vacancies
where
    experience is not null
group by
    experience
order by
    vacancies_count
'''

# результат запроса
df = pd.read_sql_query(query_7_1, connection)
df

,experience,vacancies_count,vacancies_no_salary_count,vacancies_no_salary_percent,avg_salary_from,avg_salary_to
0,Более 6 лет,1337,787,58.0,150156,197819
1,Нет опыта,7197,2142,29.0,34699,56990
2,От 3 до 6 лет,14511,8862,61.0,112544,171974
3,От 1 года до 3 лет,26152,13333,50.0,64456,97436


Здесь же мы можем наблюдать, что зарплатная вилка увеличивается при увеличении требуемого опыта кандидата.

Анализ полученных данных показывает, что зарплатные вилки, скорее, укажут в вакансиях без требуемого опыта, дальше по мере увеличения требуемого опыта количество вакансий в которых не указаны зарплатные вилки увеличивается, исключение здесь вакансии с опытом работы более 6 лет, слегка уменьшается % вакансий с неуказанной вилкой ЗП (это может быть связано с тем, что вакансий с требуемым опытом от 6 лет наименьшее количество и данные могут больше искажены по сравнению с другими категориями).

Такое можно объяснить, например, тем, что когда кандидат без опыта, то, в принципе, понятно что от такого кандидата ожидать, примерно понятно какие дополнительные затраты (затраты на обучение, на менторство и т.д.) будут после приема на работу кандидата без опыта, также понятны средние зарплаты на рынке и исходя из этого можно четко обозначить для кандидатов без опыта зарплатные вилки. Дальше же, если рассматривать кандидатов с опытом, то чем больше опыта у кандидата, тем выше может быть его специализация и профессиональные навыки (здесь речь и о хард скилах, там и о софт скилах), тем более индивидуальный подход может быть применен к кандидату, а заранее обозначение зарплатных вилок может с одной стороны привлекать много не нужных кандидатов (когда зарплатная вилка высока), либо сразу отпугивать хороших кандидатов с большим и хорошим/релевантным опытом (когда зарплатную вилку указали небольшой, хотя все же оставалось "место для маневра").

2. Рассмотрим зависимость зарплатных вилок в зависимости от необходимого опыта для Яндекса

In [527]:
query_7_2 = f'''
select
    experience,
    count(*) vacancies_count,
    sum((salary_from is null and salary_to is null)::int) vacancies_no_salary_count,
    round(100 * sum((salary_from is null and salary_to is null)::int) / count(*), 2) vacancies_no_salary_percent,
    round(avg(salary_from))::int avg_salary_from,
    round(avg(salary_to))::int avg_salary_to
from
    public.vacancies v
join
    public.employers e
        on e.id=v.employer_id
where
    e.name='Яндекс'
group by
    experience
order by
    vacancies_count
'''

df = pd.read_sql_query(query_7_2, connection)
df

,experience,vacancies_count,vacancies_no_salary_count,vacancies_no_salary_percent,avg_salary_from,avg_salary_to
0,Более 6 лет,6,6,100.0,NaN,NaN
1,От 3 до 6 лет,72,72,100.0,NaN,NaN
2,От 1 года до 3 лет,380,149,39.0,35699.0,51739.0
3,Нет опыта,1475,211,14.0,19948.0,56794.0


Здесь у Яндекса мы видим более четкую картину по зарплатным вилкам и опыту кандидатов. Ранее выдвинутая гипотеза о том, что к более опытным кандидатам применяется индивидуальный подход. Здесь мы видим, что у кандидатов без опыта самое меньшее количество вакансий без указания зарплатной вилки -- 14%, затем идут с опытом от 1 года до 3 лет с уже 39% вакансий без указания зарплатных вилок, а все остальные вакансии с опытом от 3 до 6 лет и с опытом более 6 лет идут без указания заработной платы, здесь, судя по всему, только индивидуальный подход к кандидату применяется.

3. Посмотрим на вакансии Полякова Дениса Ивановича сгруппированных по названию вакансии и посчитанным количеством регионов

In [528]:
query_7_3 = f'''
select
    v.name, count(distinct v.area_id) areas_count
from
    public.employers e
join
    public.vacancies v
        on v.employer_id=e.id
where
    e.name = 'Поляков Денис Иванович'
group by
    v.name
'''

df = pd.read_sql_query(query_7_3, connection)
df

,name,areas_count
0,Компьютерный мастер,79
1,Ученик компьютерного мастера,47


Судя по всему, Поляков Денис Иванович либо имеет региональную сеть "копьютерных мастеров" (что-то типа компьютерного мастера по объявлению и, возможно, с полу легальным оформлением всего), либо собирался её создавать. В пользу такой гипотезы говорит то, что более менее серьезные организации имеют более серьезное юридическое лицо, здесь же мы имеем ИП, которое, обычно, не предполагает создание региональной сети и приему большого количества в штат. Выглядит подозрительно :)

4. Рассмотрим среднюю предлагаемую заработную плату для DS в зависимости от опыта в Москве

In [529]:
query_7_4 = f'''
select
    experience,
    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2))::int salary_avg,
    round(avg(salary_from))::int salary_from,
    round(avg(salary_to))::int salary_to,
    min(salary_from) salary_from_min,
    max(salary_to) salary_to_max
{from_and_where_for_ds}
    and (
        salary_from is not null or
        salary_to is not null
    )
    and area_id=(
            select
                id
            from
                areas
            where
                name='Москва'
        )
group by
    experience
'''

df = pd.read_sql_query(query_7_4, connection)
df

,experience,salary_avg,salary_from,salary_to,salary_from_min,salary_to_max
0,Нет опыта,72500,45000,100000,45000,100000
1,От 1 года до 3 лет,169179,142083,210182,50000,350000
2,От 3 до 6 лет,263750,201818,351111,120000,700000


Москва демонстрирует привлекательные диапазоны предлагаемых заработных плат для DS, особенно привлекательно выглядят максимальные значения, но здесь они скорее выбросы. :)